In [ ]:
**AIM:** *To Apply Different ML methods to classify on this dataset, and combine multiple methods to get higher accuracy, approach known as ***Ensemble Machine Learning***


In [ ]:
**Overview:**

In this notebook an attempt is made at understanding features while *exploring and visualizing* the heart disease dataset. Basically I tried to show distribution of data, relations between variables and target as well as correlations between each other then did some basic model building.

**Dataset Description:**

The data includes 303 patient level features including if they have heart disease or not. Features are like;

**Age:
Sex:**
0: Female
1: Male
**Chest Pain Type:**
0: Typical Angina
1: Atypical Angina
2: Non-Anginal Pain
3: Asymptomatic
**Resting Blood Pressure:** Person's resting blood pressure.
Cholesterol: Serum Cholesterol in mg/dl
**Fasting Blood Sugar:**
0:Less Than 120mg/ml
1: Greater Than 120mg/ml
**Resting Electrocardiographic Measurement:**
0: Normal
1: ST-T Wave Abnormality
2: Left Ventricular Hypertrophy
Max Heart Rate Achieved: Maximum Heart Rate Achieved
**Exercise Induced Angina:**
1: Yes
0: No 
**etc**


After some usual exploratory data analysis (EDA) I noticed some of the results doesn't make sense, and then I found this topic here. The ultimate guide to this dataset!, his points made sense so I decided to use this dataset after inspecting it: **Heart Disease Cleveland UCI.** from the official repository

Okay, **let's START**

In [ ]:
#1: Loading the Dataset

In [ ]:
# Import all the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
import math
import random
import os
import time
from numpy import interp

# Disabling warnings:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
# Apply some Styles:

cust_palt = [
    '#111d5e', '#c70039', '#f37121', '#ffbd69', '#ffc93c'
]
plt.style.use('ggplot')

In [ ]:
# Seeding:

def seed_all(seed):
    
    ''' A function to seed everything for getting stable results and reproducibility'''
    
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed = 42    
seed_all(seed)

x=144
y=123

In [ ]:
# Reading the csv file:
from google.colab import files
uploaded = files.upload()
#train = pd.read_csv("heartDataset.csv")